## Web Page Analytics

Caching all the items in redis will consume memory. We want to just cache the commonly viewed items.
We can do this by keeping track of the view counts, and just keeping the top 10,000 viewed items by trimming the rest.

In [31]:
import time


def update_token(conn, token, user, item=None):
    timestamp = time.time()
    conn.hset("login:", token, user)
    conn.zadd("recent:", {token: timestamp})
    if item:
        conn.zadd("viewed:" + token, {item: timestamp})
        conn.zremrangebyrank("viewed:" + token, 0, -26)
        # Keep record of count of the items viewed.
        # Note that we are decrementing the value, this is because ZSET are stored in ascending order.
        conn.zincrby("viewed:", -1, item)

In [19]:
def rescale_viewed(conn):
    # Remove any items not in the the top 20,000 viewed items.
    conn.zremrangebyrank("viewed:", 20_000, -1)

    # Rescale all counts to be 1/2 of what they were before..
    conn.zinterstore("viewed:", {"viewed:": 0.5})


def rescale_viewed_job(conn):
    while not QUIT:
        rescale_viewed(conn)

        # Do it again in 5 minutes.
        time.sleep(300)

In [20]:
def can_cache(conn, request):
    item_id = extract_item_id(request)

    # Check whether the page can be statically cached and whether
    # this is an item page.
    if not item_id or is_dynamic(request):
        return False

    # Get the rank of the item.
    rank = conn.zrank("viewed:", item_id)

    # Return whether the item has high enough view count to be cached.
    return rank is not None and rank < 10_000

In [21]:
import utils

conn = utils.connect()
conn.ping()

True

In [22]:
conn.zadd("hello", {"john": 1, "alice": 2})

0

In [23]:
conn.zrange("hello", 0, -1, withscores=True)

[('john', 1.0), ('alice', 2.0)]

In [24]:
conn.zinterstore(
    "hello", {"hello": 0.5}
)  # this operation halves the score of the zset "hello"

2

In [25]:
conn.zrange("hello", 0, -1, withscores=True)

[('john', 0.5), ('alice', 1.0)]

In [54]:
for i in range(10):
    update_token(conn, "token123", "user123", "item123")

In [56]:
conn.zrange("viewed:", 0, -1, withscores=True)

[('item123', -15.1875)]

In [57]:
rescale_viewed(conn)

In [58]:
conn.zrange("viewed:", 0, -1, withscores=True)

[('item123', -7.59375)]